In [1]:
import pandas as pd

In [113]:
df = pd.read_csv('lectionary.tsv', sep='\t', header=1)

In [114]:
df = df.rename(lambda x: str.lower(x).replace(" ", "_"), axis='columns')

In [115]:
df['month'] = df['date'].apply(lambda x: int(x.split('-')[0]))
df['day'] = df['date'].apply(lambda x: int(x.split('-')[1]))

In [116]:
df = df.set_index(['date'])

In [117]:
# Add in Psalm
psalm_cols = [col for col in df if col.endswith('psalm')]
for col in psalm_cols:
    df[col] = df[col].apply(lambda x: 'Psalm ' + x)

In [118]:
import re

class Passage():
    def __init__(self, bcp_format):
        self.bcp_format = bcp_format
        self.is_entire_book = False
        self.parse()
    
    def parse(self):
        pattern = re.compile('^(\d? ?[A-Za-z ]+) ?(.*)')
        match = pattern.match(self.bcp_format)
        self.book = match.group(1).strip()        
        self.verses = match.group(2).strip()

        if self.verses == '':
            self.is_entire_book = True
        
        # Look for an em dash
        if '—' in self.verses:
            self.verses = self.verses.replace('—', '-')
            if self.verses.endswith(' end'):
                self.verses = self.verses.replace(' end', ':1-end')
    
        if ' & ' in self.verses:
            self.verses = self.verses.replace(' & ', ', ')
    
    def __str__(self):
        if self.is_entire_book:
            return self.book
        else:
            return "%s %s" % (self.book, self.verses)

In [119]:
df

,holy_day,mp_psalm,mp_lesson_1,mp_lesson_2,ep_psalm,ep_lesson_1,ep_lesson_2,month,day
date,,,,,,,,,
01-01,Holy Name,"Psalm 1, 2",Gen 1,John 1:1-28,"Psalm 3, 4",Gal 1,Luke 2:8-21,1,1
01-02,NaN,"Psalm 5, 6",Gen 2,John 1:29-end,Psalm 7,Jer 1,Gal 2,1,2
01-03,NaN,Psalm 9,Gen 3,John 2,Psalm 10,Jer 2 † 1-22,Gal 3,1,3
01-04,NaN,"Psalm 8, 11",Gen 4,John 3:1-21,"Psalm 15, 16",Jer 3,Gal 4,1,4
01-05,NaN,"Psalm 12, 13, 14",Gen 5,John 3:22-end,Psalm 17,Jer 4,Gal 5,1,5
...,...,...,...,...,...,...,...,...,...
12-27,John,Psalm 139,Rev 19,John 21:9-25,"Psalm 141, 142",Song of Songs 4,Luke 22:54-end,12,27
12-28,Holy Innocents,Psalm 140,Jer 31:1-17,Rev 20,Psalm 143,Song of Songs 5,Luke 23:1-25,12,28
12-29,NaN,Psalm 144,Wisdom 9,Rev 21:1-14,Psalm 145,Song of Songs 6,Luke 23:26-49,12,29


In [120]:
# For now, strip out the abbreviated readings
reading_cols = [col for col in df if col.startswith('mp') or col.startswith('ep')]
for col in reading_cols:
    df[col] = df[col].apply(lambda x: str(Passage(x.split(' † ')[0])))

In [121]:
df.head(90)

,holy_day,mp_psalm,mp_lesson_1,mp_lesson_2,ep_psalm,ep_lesson_1,ep_lesson_2,month,day
date,,,,,,,,,
01-01,Holy Name,"Psalm 1, 2",Gen 1,John 1:1-28,"Psalm 3, 4",Gal 1,Luke 2:8-21,1,1
01-02,NaN,"Psalm 5, 6",Gen 2,John 1:29-end,Psalm 7,Jer 1,Gal 2,1,2
01-03,NaN,Psalm 9,Gen 3,John 2,Psalm 10,Jer 2,Gal 3,1,3
01-04,NaN,"Psalm 8, 11",Gen 4,John 3:1-21,"Psalm 15, 16",Jer 3,Gal 4,1,4
01-05,NaN,"Psalm 12, 13, 14",Gen 5,John 3:22-end,Psalm 17,Jer 4,Gal 5,1,5
...,...,...,...,...,...,...,...,...,...
03-26,NaN,"Psalm 61, 62",Exod 33,Matt 26:57-end,"Psalm 65, 67",Prov 24,Eph 5:18-end,3,26
03-27,NaN,Psalm 68:1-18,Exod 34,Matt 27:1-26,Psalm 68:19-36,Prov 25,Eph 6:1-9,3,27
03-28,NaN,Psalm 69:1-18,Exod 35,Matt 27:27-56,Psalm 69:19-38,Prov 26,Eph 6:10-end,3,28


In [123]:
df.to_json(r'lectionary.json', orient='index', indent=2)